In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, GRU, Flatten
from sklearn.preprocessing import StandardScaler
from keras import ops
import keras

In [75]:
# Завантажимо файл і подивимось на вміст
path = r'C:\PythonProject\PythonCoreCourse\PythonDataSience\project\internet_service_churn.csv'
data = pd.read_csv(path)
data.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


In [76]:
# Згідно рекомендацій попередьної частини зробим потрібні перетворення по  очищенню данних датасета
data = data.fillna({'reamining_contract':data['reamining_contract'].min()}).dropna()
data = data.iloc[:, 1:]
data.isnull().sum(), data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 71893 entries, 0 to 72273
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   is_tv_subscriber             71893 non-null  int64  
 1   is_movie_package_subscriber  71893 non-null  int64  
 2   subscription_age             71893 non-null  float64
 3   bill_avg                     71893 non-null  int64  
 4   reamining_contract           71893 non-null  float64
 5   service_failure_count        71893 non-null  int64  
 6   download_avg                 71893 non-null  float64
 7   upload_avg                   71893 non-null  float64
 8   download_over_limit          71893 non-null  int64  
 9   churn                        71893 non-null  int64  
dtypes: float64(4), int64(6)
memory usage: 6.0 MB


(is_tv_subscriber               0
 is_movie_package_subscriber    0
 subscription_age               0
 bill_avg                       0
 reamining_contract             0
 service_failure_count          0
 download_avg                   0
 upload_avg                     0
 download_over_limit            0
 churn                          0
 dtype: int64,
 None)

In [77]:
# Розділимо датасет, відберемо ознаки і таргет-колонку
X = data.iloc[:, :-1]
y = data.churn

# зробимо тренувальну і тестову вибірки даних, подивмось на їх розмірності для контролю.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((57514, 9), (14379, 9), (57514,), (14379,))

In [79]:
# One-hot кодування таргету, можливо можно то і не робить використавши 1 нейрон для бинарної класифікації, але зробим загальний випадок
y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)

In [12]:
X_train.shape, y_train.shape

((57514, 9), (57514, 2))

In [23]:

# Для застосування нашої моделі внесемо до неї шар попередної нормалізації ознак розрахований на нашому датасеті


class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())
        

std_layer = Standardization(input_shape=[9, ])
std_layer.adapt(np.array(X))

C:\Users\Admin\AppData\Local\Temp\ipykernel_956\179999217.py:26: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  std_layer = Standardization(input_shape=[9, ])


In [10]:
# статистичні констатнти для стандартизації вхідних даних
std_layer.means_, std_layer.stds_


(array([[ 0.8152254 ,  0.3340659 ,  2.45548593, 19.01684448,  0.49761242,
          0.27567357, 43.6899114 ,  4.19207572,  0.20871295]]),
 array([[ 0.3881146 ,  0.47166288,  2.02684238, 13.18844911,  0.66575045,
          0.81852776, 63.40552204,  9.81882807,  0.99964011]]))

In [81]:
# модель без шару сдандартизації
model_NN1 = keras.Sequential()
model_NN1.add(keras.layers.Flatten(input_shape=(9, )))
model_NN1.add(keras.layers.Dense(128, activation='tanh'))
model_NN1.add(keras.layers.Dense(256, activation='tanh'))
model_NN1.add(keras.layers.Dense(2, activation='sigmoid'))
                              
model_NN1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [83]:
# Тренування моделі 
history_NN1 = model_NN1.fit(
    X_train, y_train, epochs=25,
    validation_split=0.2,)
   

Epoch 1/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8722 - loss: 0.3420 - val_accuracy: 0.9039 - val_loss: 0.2748
Epoch 2/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9126 - loss: 0.2613 - val_accuracy: 0.9156 - val_loss: 0.2495
Epoch 3/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9204 - loss: 0.2433 - val_accuracy: 0.9166 - val_loss: 0.2432
Epoch 4/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9233 - loss: 0.2356 - val_accuracy: 0.9226 - val_loss: 0.2371
Epoch 5/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9248 - loss: 0.2301 - val_accuracy: 0.9225 - val_loss: 0.2334
Epoch 6/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9241 - loss: 0.2300 - val_accuracy: 0.9225 - val_loss: 0.2388
Epoch 7/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9251 - loss: 0.2276 - val_accuracy: 0.9226 - val_loss: 0.2390
Epoch 8/25
1438/1438 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9277 - loss: 0.2237 - 

In [41]:
# збережемо в окремому обьеті константи для стандартизації вхвдних даних
import joblib
means = std_layer.means_
stds = std_layer.stds_
eps = keras.backend.epsilon()
values = (means, stds, eps)
#(inputs - means) / (stds + eps)
joblib.dump(values, r'C:\PythonProject\values.pkl')

['C:\\PythonProject\\values.pkl']

In [84]:
# збережемо просту модель
model_NN1.save(r'C:\PythonProject\model_NN1.keras')

In [55]:
# далі йдуть нецікаві тести відповідності класифікатора
np.array(X.iloc[0, :]), y[0]

(array([ 1.  ,  0.  , 11.95, 25.  ,  0.14,  0.  ,  8.4 ,  2.3 ,  0.  ]), 0)

In [56]:
histoty_aaa = model_NN1.predict(X)

2247/2247 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


In [60]:
histoty_aaa[:6]


array([[9.9039620e-01, 1.1777583e-02],
       [7.3790306e-04, 9.9933016e-01],
       [7.2477631e-02, 9.2341387e-01],
       [4.2939270e-03, 9.9594468e-01],
       [6.0910493e-04, 9.9941921e-01],
       [9.2520076e-01, 8.4866785e-02]], dtype=float32)

In [62]:
data

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,0,0,8.22,0,0.00,0,0.0,0.0,0,1
2,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,0,0,6.87,21,0.00,1,0.0,0.0,0,1
4,0,0,6.39,0,0.00,0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...
72269,1,1,0.09,0,1.25,0,0.0,0.0,0,1
72270,1,0,0.06,1,1.63,0,0.8,0.0,0,1
72271,1,0,0.02,0,2.19,0,1.5,0.2,0,1
72272,0,0,0.01,0,0.72,0,0.0,0.0,0,1


In [85]:
inputs=np.array(X.iloc[0, :])
a = (inputs - means) / (stds + eps)
model_NN1.predict(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


array([[0.98797935, 0.01087761]], dtype=float32)

In [89]:
np.array(X.iloc[4, :])

array([0.  , 0.  , 6.39, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])